In [40]:
import os
import glob
import h5py

def copy_items(name, obj, destination_group):
    if isinstance(obj, h5py.Group):
        # Check if the group already exists in the destination H5 file
        if name not in destination_group:
            # Create a new group in the destination H5 file
            new_group = destination_group.create_group(name)
            # Recursively copy groups and datasets from the current group
            obj.visititems(lambda subname, subobj: copy_items(subname, subobj, new_group))
    elif isinstance(obj, h5py.Dataset):
        # Append datasets with the same name in the destination H5 file
        if name in destination_group:
            existing_dataset = destination_group[name]
            existing_shape = existing_dataset.shape
            existing_dataset.resize((existing_shape[0] + obj.shape[0],) + existing_shape[1:])
            existing_dataset[-obj.shape[0]:] = obj[:]
        else:
            # Create a new dataset in the destination H5 file with chunks
            chunks = (1,) + obj.shape[1:]
            destination_group.create_dataset(name, shape=obj.shape, dtype=obj.dtype, chunks=chunks, maxshape=(None,) + obj.shape[1:], data=obj[:])

def concatenate_h5_in_folder(output_file, folder='ML_ana_h5'):
    # Get all H5 files in the specified folder
    files_to_concatenate = glob.glob(os.path.join(folder, '*.h5'))

    # Create a new H5 file for concatenation
    with h5py.File(output_file, 'w') as output_h5:
        for file_path in files_to_concatenate:
            # Open each input H5 file
            with h5py.File(file_path, 'r') as input_h5:
                # Recursively copy items from the input H5 file to the output H5 file
                input_h5.visititems(lambda name, obj: copy_items(name, obj, output_h5))

# Example usage:
output_file_path = 'combined_out.h5'
concatenate_h5_in_folder(output_file_path)


AttributeError: 'File' object has no attribute 'shape'